In [170]:
import pandas as pd
import numpy as np
import re

In [171]:
test = pd.read_csv("champ1_test_.csv", encoding='utf-8')

In [172]:
test.columns

Index([u'_ID_', u'_CITY_', u'_ADRS_', u'_OBJ_TYPE_', u'_TTL_S_', u'_F1_S_',
       u'_F1_U_', u'_FC_S_', u'_FC_U_', u'_F0_S_', u'_F0_U_', u'_FA_S_',
       u'_FA_U_', u'_F2_S_', u'_F2_U_', u'_F3_S_', u'_F3_U_', u'_AREA_',
       u'_CHARACT_', u'_LINE_', u'_METRO_', u'_ROUND_', u'_FOOT_TRAF_',
       u'_IS_PRKNG_', u'_IS_WIN_', u'_IS_SEP_ENT_', u'_IS_VENT_', u'_DECOR_',
       u'_IS_COM_', u'_F1_H_', u'_DATE_'],
      dtype='object')

## 'CITY'

In [175]:
pattern_city = ur'[(г)(д)(г\.п)(с)(пгт)(пос)(р\.п)]+(\.| )[ \w-]*'

def parse_city(x):
    for i in re.finditer(pattern_city, x):
        res = x[i.end()-1:].split(u'.')[-1].strip()
        return res
    return x.strip()

In [176]:
city_dict = {}
for old, new in zip(test._CITY_.unique(), pd.Series(test._CITY_.unique()).apply(lambda x: parse_city(x))):
    city_dict[old] = new

In [178]:
test['CITY'] = test._CITY_.apply(lambda x: city_dict[x])
len(test['CITY'].unique())

787

## '_OBJ_TYPE_' ##

In [179]:
for value in test._OBJ_TYPE_.unique():
    print value

помещение
отдельно стоящее здание
Нежилое помещение (магазин) с крыльцом
Нежилое помещение, назначение : нежилое, общая площадь 76,4 кв.м.
Здание, назначение: нежилое здание, 2-этажное, общая площадь 725,9 кв.м, лит. А.А1
Нежилое помещение, назначение : нежилое, общая площадь 41,9 кв.м., этаж 1
Встроенные помещения, назначение: нежилое, площадь 170,8 кв.м., этаж цоколный
Часть здания корпуса № 70, назначение: нежилое, общая площадь 145,2 кв.м., номера на поэтажном плане 30,33
Нежилое помещение, назначение: нежилое, общая площадь 303,2 кв.м., лит.3, этаж 3, номера на поэтажном плане 34-40-42


In [180]:
map_obj_type = [
    u'помещение',
    u'здание',
    u'помещение',
    u'помещение',
    u'здание',
    u'помещение',
    u'помещение',
    u'помещение',
    u'помещение'
]

In [181]:
map_obj_type_good = [
    True,
    True,
    False,
    False,
    False,
    False,
    False,
    False,
    False
]

In [182]:
obj_type_dict = {}
for x,y,z in zip(test._OBJ_TYPE_.unique(), map_obj_type, map_obj_type_good):
    obj_type_dict[x] = [y, z]

In [183]:
test['OBJ_TYPE'] = test._OBJ_TYPE_.apply(lambda x: obj_type_dict[x][0])
test['OBJ_TYPE_AUTO'] = test._OBJ_TYPE_.apply(lambda x: obj_type_dict[x][1])

## SQUARE ##

In [184]:
def to_float(line):
    try:
        return float(line.encode('ascii','ignore').replace(',','.'))
    except:
        return 0

In [185]:
test['TTL_S'] = test._TTL_S_.apply(lambda x: to_float(x))

In [186]:
test['F1_S'] = test[u'_F1_S_'].apply(lambda x: to_float(x))
test['FC_S'] = test[u'_FC_S_'].apply(lambda x: to_float(x))
test['F0_S'] = test[u'_F0_S_'].apply(lambda x: to_float(x))
test['FA_S'] = test[u'_FA_S_'].apply(lambda x: to_float(x))
test['F2_S'] = test[u'_F2_S_'].apply(lambda x: to_float(x))
test['F3_S'] = test[u'_F3_S_'].apply(lambda x: to_float(x))

## USE

In [187]:
dict_use_types = {
    u'технич': u'техническое',
    u'склад': u'складское',
    u'торгов': u'торговое',
    u'офис': u'офисное',
    u'жил': u'жилое'
}

def parse_use_types(desc=u''):
    if type(desc) == unicode:
        for t in dict_use_types:
            if desc.find(t) >= 0:
                return dict_use_types[t]
    return np.nan

In [188]:
test['F1_U'] = test[u'_F1_U_'].apply(lambda x: parse_use_types(x))
test['FC_U'] = test[u'_FC_U_'].apply(lambda x: parse_use_types(x))
test['F0_U'] = test[u'_F0_U_'].apply(lambda x: parse_use_types(x))
test['FA_U'] = test[u'_FA_U_'].apply(lambda x: parse_use_types(x))
test['F2_U'] = test[u'_F2_U_'].apply(lambda x: parse_use_types(x))
test['F3_U'] = test[u'_F3_U_'].apply(lambda x: parse_use_types(x))

## CHARACT

In [189]:
dict_CHARACT = [
    (u'магистраль', u'основная магистраль'),
    (u'основная', u'основная улица'),
    (u'второстепенная', u'второстепенная улица'),
    (u'центр', u'центр')
]

def parse_CHARACT(desc=u''):
    if type(desc) == unicode:
        for t,v in dict_CHARACT:
            if desc.find(t) >= 0:
                return v
    return np.nan

In [190]:
test[u'CHARACT'] = test[u'_CHARACT_'].apply(lambda x: parse_CHARACT(x))

## METRO ##

In [191]:
dict_METRO = [
    (u'транспортная', u'транспортная доступность'),
    (u'пешеходная', u'пешеходная доступность'),
    (u'с метро', u'рядом с метро')
]

def parse_METRO(desc=u''):
    if type(desc) == unicode:
        for t,v in dict_METRO:
            if desc.find(t) >= 0:
                return v
    return np.nan

In [192]:
test[u'METRO'] = test[u'_METRO_'].apply(lambda x: parse_METRO(x))

## IS_PRKNG ##

In [193]:
dict_PRKNG = [
    (u'городская ', u'городская парковка'),
    (u'выделенная ', u'выделенная парковка'),
    (u'да', u'городская парковка'),
    (u'нет', u'нет')
]

def parse_PRKNG(desc=u''):
    if type(desc) == unicode:
        for t,v in dict_PRKNG:
            if desc.find(t) >= 0:
                return v
    return np.nan

In [195]:
test[u'IS_PRKNG'] = test[u'_IS_PRKNG_'].apply(lambda x: parse_PRKNG(x)).value_counts()

In [196]:
test['F1_H'] = test[u'_F1_H_'].apply(lambda x: to_float(x))

In [197]:
 test[[u'_ID_', u'CITY', u'_ADRS_', u'OBJ_TYPE', u'TTL_S', u'F1_S',
       u'F1_U', u'FC_S', u'FC_U', u'F0_S', u'F0_U', u'FA_S',
       u'FA_U', u'F2_S', u'F2_U', u'F3_S', u'F3_U', u'_AREA_',
       u'CHARACT', u'_LINE_', u'METRO', u'_ROUND_', u'_FOOT_TRAF_',
       u'IS_PRKNG', u'_IS_WIN_', u'_IS_SEP_ENT_', u'_IS_VENT_', u'_DECOR_',
       u'_IS_COM_', u'F1_H']].to_csv("champ1_test_clean.csv", encoding='utf-8', index=False)

In [198]:
test.columns

Index([         u'_ID_',        u'_CITY_',        u'_ADRS_',    u'_OBJ_TYPE_',
             u'_TTL_S_',        u'_F1_S_',        u'_F1_U_',        u'_FC_S_',
              u'_FC_U_',        u'_F0_S_',        u'_F0_U_',        u'_FA_S_',
              u'_FA_U_',        u'_F2_S_',        u'_F2_U_',        u'_F3_S_',
              u'_F3_U_',        u'_AREA_',     u'_CHARACT_',        u'_LINE_',
             u'_METRO_',       u'_ROUND_',   u'_FOOT_TRAF_',    u'_IS_PRKNG_',
            u'_IS_WIN_',  u'_IS_SEP_ENT_',     u'_IS_VENT_',       u'_DECOR_',
            u'_IS_COM_',        u'_F1_H_',        u'_DATE_',          u'CITY',
            u'OBJ_TYPE', u'OBJ_TYPE_AUTO',         u'TTL_S',          u'F1_S',
                u'FC_S',          u'F0_S',          u'FA_S',          u'F2_S',
                u'F3_S',          u'F1_U',          u'FC_U',          u'F0_U',
                u'FA_U',          u'F2_U',          u'F3_U',       u'CHARACT',
               u'METRO',      u'IS_PRKNG',          

In [267]:
req = [
    u'F1_U',              
    u'FC_U',        
    u'F0_U',              
    u'FA_U',        
    u'F2_U',              
    u'F3_U'
]
levels = [
    u'1 этаж',              
    u'цоколь',        
    u'подвал',              
    u'антресоль',        
    u'2 этаж',              
    u'3 этаж'
]
def is_building(x):
    c = 0
    for r in req:
        if not pd.isnull(x[r]):
            c+=1
    return c > 1

def get_type(x):
    for r in req:
        if not pd.isnull(x[r]):
            return x[r]
    return np.nan

def get_level(x):
    for r,level in zip(req, levels):
        if not pd.isnull(x[r]):
            return level
    return np.nan

In [268]:
print is_building(test.iloc[0,:])
print get_type(test.iloc[0,:])
print get_level(test.iloc[0,:])

False
торговое
1 этаж


In [269]:
test['is_building'] = test.apply(lambda x: is_building(x), axis=1)

In [270]:
buildings = test[test['is_building']]
non_buildings = test[~test['is_building']]

In [271]:
non_buildings['type'] = non_buildings.apply(get_type, axis=1)
non_buildings['level'] = non_buildings.apply(get_level, axis=1)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [272]:
non_buildings = non_buildings[~non_buildings['type'].isnull()]
# non_buildings

In [273]:
 buildings[[u'_ID_', u'CITY', u'_ADRS_', u'OBJ_TYPE', u'TTL_S', u'F1_S',
       u'F1_U', u'FC_S', u'FC_U', u'F0_S', u'F0_U', u'FA_S',
       u'FA_U', u'F2_S', u'F2_U', u'F3_S', u'F3_U', u'_AREA_',
       u'CHARACT', u'_LINE_', u'METRO', u'_ROUND_', u'_FOOT_TRAF_',
       u'IS_PRKNG', u'_IS_WIN_', u'_IS_SEP_ENT_', u'_IS_VENT_', u'_DECOR_',
       u'_IS_COM_', u'F1_H']].to_csv("champ1_test_buildings.csv", encoding='utf-8', index=False)

In [276]:
 cols = [u'_ID_', u'CITY', u'_ADRS_', u'OBJ_TYPE', u'TTL_S', 'type', 'level', u'_AREA_',
       u'CHARACT', u'_LINE_', u'METRO', u'_ROUND_', u'_FOOT_TRAF_',
       u'IS_PRKNG', u'_IS_WIN_', u'_IS_SEP_ENT_', u'_IS_VENT_', u'_DECOR_',
       u'_IS_COM_', u'F1_H']
    
non_buildings[non_buildings['type'] == u'торговое'][cols].to_csv("champ1_test_shops.csv", encoding='utf-8', index=False)
non_buildings[non_buildings['type'] == u'офисное'][cols].to_csv("champ1_test_office.csv", encoding='utf-8', index=False)
non_buildings[non_buildings['type'] == u'складское'][cols].to_csv("champ1_test_warehouse.csv", encoding='utf-8', index=False)
non_buildings[non_buildings['type'] == u'техническое'][cols].to_csv("champ1_test_technical.csv", encoding='utf-8', index=False)

In [278]:
non_buildings[u'OBJ_TYPE'].value_counts()

помещение    3993
здание        525
Name: OBJ_TYPE, dtype: int64

In [274]:
non_buildings['type'].value_counts()

торговое       2849
офисное        1339
складское       251
техническое      79
Name: type, dtype: int64

In [280]:
test_cols = [u'_ID_', u'TTL_S', 'type', u'_AREA_',
       u'_LINE_', u'METRO', u'_FOOT_TRAF_',
       u'IS_PRKNG', u'_IS_WIN_', u'_IS_SEP_ENT_', u'_DECOR_',
       u'_IS_COM_', u'F1_H']
non_buildings[test_cols]

,_ID_,TTL_S,type,_AREA_,_LINE_,METRO,_FOOT_TRAF_,IS_PRKNG,_IS_WIN_,_IS_SEP_ENT_,_DECOR_,_IS_COM_,F1_H
0,0,184.7,торговое,центр,первая линия,рядом с метро,очень высокий,NaN,да,да,рабочее состояние,все,2.70
1,1,284.8,торговое,спальный район,первая линия,пешеходная доступность,низкий,NaN,да,да,рабочее состояние,все,0.00
2,2,59.6,торговое,деловой центр,первая линия,транспортная доступность,средний,NaN,да,да,рабочее состояние,все,2.75
4,4,73.3,торговое,центр,первая линия,пешеходная доступность,средний,NaN,нет,да,рабочее состояние,все,4.00
5,5,35.1,торговое,центр,первая линия,пешеходная доступность,средний,NaN,нет,да,рабочее состояние,все,2.75
6,6,190.6,торговое,центр,первая линия,пешеходная доступность,низкий,NaN,нет,да,евроремонт,все,4.10
7,7,140.6,торговое,спальный район,первая линия в глубине,транспортная доступность,низкий,NaN,да,да,евроремонт,все,3.59
8,8,88.5,торговое,спальный район,первая линия,рядом с метро,средний,NaN,нет,да,рабочее состояние,все,2.58
9,9,142.7,торговое,центр,первая линия,рядом с метро,средний,NaN,да,да,рабочее состояние,все,2.80
10,10,141.5,торговое,центр,первая линия,транспортная доступность,низкий,NaN,нет,да,без отделки,все,3.28
